In [2]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np

In [13]:
import pandas as pd

# Specify the full file path
file_path = r"C:\Users\saima\Downloads\GenContingencyAllGen.xlsx" 
df = pd.read_excel(file_path)

print(df.head())

   Run #  Pm02  Pm03  factor5  factor6  factor8  f_nadir(Hz)  RoCof(Hz/s)  \
0      1   143    65      0.9      0.9      0.9    59.321788    -0.922567   
1      2   148    65      0.9      0.9      0.9    59.360917    -0.876443   
2      3   153    65      0.9      0.9      0.9    59.400090    -0.827881   
3      4   158    65      0.9      0.9      0.9    59.439296    -0.768516   
4      5   163    65      0.9      0.9      0.9    59.478582    -0.718783   

   InputPg1(MW) InputQg1(MVar)  ... InputQg2(MVar)  InputPg3(MW)  \
0     41.117377      41.158379  ...     108.862559     21.856561   
1     38.649959      38.686376  ...     112.537104     21.857274   
2     36.181663      36.213798  ...     116.210692     21.857992   
3     33.712519      33.740681  ...     119.883272     21.858614   
4     31.242544      31.267048  ...     123.554810     21.859126   

   InputQg3(MVar)  Pload5_og(MW)  Qload5_og(MVar)  Pload6_og(MW)  \
0       43.713063     113.317420        45.433809      83.36

In [15]:
# Features and targets
X = df.drop(columns=["f_nadir(Hz)", "RoCof(Hz/s)", "Run #", "gen"])
y = df[["f_nadir(Hz)", "RoCof(Hz/s)"]]

In [23]:
import pandas as pd
import numpy as np

# Check which columns are object or string type
non_numeric_cols = X.select_dtypes(include='object').columns
print("Non-numeric columns:", non_numeric_cols)

# Clean and convert each one
for col in non_numeric_cols:
    print(f"\nBefore cleaning {col}:")
    print(X[col].unique()[:5])  # show first few unique values

    # Try to split malformed strings and keep the correct part
    X[col] = X[col].astype(str).str.extract(r'([+-]?[0-9]*\.?[0-9]+(?:[eE][+-]?[0-9]+)?)')[0]

    # Convert to numeric (coerce errors to NaN)
    X[col] = pd.to_numeric(X[col], errors='coerce')

    print(f"After cleaning {col}:")
    print(X[col].unique()[:5])


Non-numeric columns: Index(['InputQg1(MVar)', 'InputPg2(MW)'], dtype='object')

Before cleaning InputQg1(MVar):
[41.15837875 38.68637589 36.21379788 33.74068116 31.26704821]
After cleaning InputQg1(MVar):
[41.15837875 38.68637589 36.21379788 33.74068116 31.26704821]

Before cleaning InputPg2(MW):
[36.2875198 37.51236788 38.73689726 39.9610906 41.18493652]
After cleaning InputPg2(MW):
[36.2875198  37.51236788 38.73689726 39.9610906  41.18493652]


In [25]:
X = X.dropna()
y = y.loc[X.index]  # align y with X after dropping

In [27]:
from sklearn.preprocessing import StandardScaler

x_scaler = StandardScaler()
y_scaler = StandardScaler()

X_scaled = x_scaler.fit_transform(X)
y_scaled = y_scaler.fit_transform(y)

In [29]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

In [31]:
# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [33]:
# Define the base NN model
class BaseNN(nn.Module):
    def __init__(self):
        super(BaseNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(X_train.shape[1], 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )

    def forward(self, x):
        return self.net(x)

In [35]:
# Instantiate and train the model
model = BaseNN()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [37]:
# Training loop
n_epochs = 1000
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.4f}")

Epoch 100/1000, Loss: 0.6217
Epoch 200/1000, Loss: 0.5253
Epoch 300/1000, Loss: 0.4973
Epoch 400/1000, Loss: 0.4758
Epoch 500/1000, Loss: 0.4606
Epoch 600/1000, Loss: 0.4467
Epoch 700/1000, Loss: 0.4357
Epoch 800/1000, Loss: 0.4270
Epoch 900/1000, Loss: 0.4192
Epoch 1000/1000, Loss: 0.4124


In [39]:
# Evaluation
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    predictions = y_scaler.inverse_transform(predictions.numpy())
    y_test_actual = y_scaler.inverse_transform(y_test.numpy())

rmse_f_nadir = np.sqrt(mean_squared_error(y_test_actual[:, 0], predictions[:, 0]))
rmse_rocof = np.sqrt(mean_squared_error(y_test_actual[:, 1], predictions[:, 1]))

print(f"RMSE - f_nadir: {rmse_f_nadir:.4f} Hz")
print(f"RMSE - RoCof: {rmse_rocof:.4f} Hz/s")


RMSE - f_nadir: 0.1128 Hz
RMSE - RoCof: 0.1453 Hz/s
